In [3]:
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from time import sleep
from time import time
import random

In [13]:
# random_sleep = round(random.uniform(2, 3), 2)
def scraping_airbnb(url):
    
    options = webdriver.ChromeOptions()
    options.add_argument("--disable-search-engine-choice-screen")
    options.add_argument("--headless")
    options.add_argument('--lang=en')
    options.add_argument('--disable-geolocation')
    driver = webdriver.Chrome(options=options)
    driver.get(url)

    # Esperar que la página cargue y gestionar popup de traducción
    try:
        WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//div[@class='c1lbtiq8 atm_mk_stnw88 atm_9s_1txwivl atm_fq_1tcgj5g atm_wq_kb7nvz atm_tk_1tcgj5g dir dir-ltr']"))
        ).click()
    except:
        pass
    random_sleep = round(random.uniform(2, 3), 2)
    sleep(random_sleep)
    title = extract_title(driver)
    sleep(random_sleep)
    rating = extract_rating(driver)
    sleep(random_sleep)
    number_reviews = extract_number_reviews(driver)
    sleep(random_sleep)
    guest_favorite = extract_guest_favorite(driver)
    sleep(random_sleep)
    type_host = extract_type_host(driver)
    sleep(random_sleep)
    data_list, hosting_time = extract_data_list(driver)
    sleep(random_sleep)
    all_reviews = extract_reviews(driver, number_reviews)
    
    return guest_favorite, rating, number_reviews, type_host, hosting_time, all_reviews, title, data_list
    
def extract_title(driver):
    try:
        return driver.find_element(By.XPATH, "//div[@class = '_1czgyoo']").text
    except:
        return np.nan
    
def extract_number_reviews(driver):
    try:
        number_reviews = driver.find_element(By.XPATH, '//div[@class="r16onr0j atm_c8_vvn7el atm_g3_k2d186 atm_fr_1vi102y atm_gq_myb0kj atm_vv_qvpr2i atm_c8_sz6sci__14195v1 atm_g3_17zsb9a__14195v1 atm_fr_kzfbxz__14195v1 atm_gq_idpfg4__14195v1 dir dir-ltr"]').text
    except:
        try:
            number_reviews = driver.find_element(By.XPATH, '//div[@dir="ltr"]').text
            number_reviews = number_reviews.split(" ")[2]
        except:
            number_reviews = np.nan
    return number_reviews

def extract_guest_favorite(driver):
    guest_favorite = False
    guest = "no"
    try:
        guest = driver.find_element(By.XPATH, '//div[@class="lbjrbi0 atm_le_1y44olf atm_lk_1y44olf atm_ll_1y44olf dir dir-ltr"]').text
    except:
        pass
    if guest != "no":
        guest_favorite = True
    return guest_favorite

def extract_rating(driver):
    try:
        rating = driver.find_element(By.XPATH, '//div[@class="r1lutz1s atm_c8_o7aogt atm_c8_l52nlx__oggzyc dir dir-ltr"]').text
    except:
        try:
            random_sleep = round(random.uniform(2, 3), 2)
            sleep(random_sleep)
            rating = driver.find_element(By.XPATH, '//div[@class="a8jhwcl atm_c8_vvn7el atm_g3_k2d186 atm_fr_1vi102y atm_9s_1txwivl atm_ar_1bp4okc atm_h_1h6ojuz atm_cx_t94yts atm_le_14y27yu atm_c8_sz6sci__14195v1 atm_g3_17zsb9a__14195v1 atm_fr_kzfbxz__14195v1 atm_cx_1l7b3ar__14195v1 atm_le_1l7b3ar__14195v1 dir dir-ltr"]').text
        except:
            rating = np.nan
    return rating

def extract_type_host(driver):
    try:
        type_host = driver.find_element(By.XPATH, '//li[@class="l7n4lsf atm_9s_1o8liyq_keqd55 dir dir-ltr"]').text
    except:
        type_host = np.nan
    return type_host

def extract_data_list(driver):
    try:
        complete_data_list_primer = driver.find_elements(By.XPATH, '//div[@class="o1kjrihn atm_c8_km0zk7 atm_g3_18khvle atm_fr_1m9t47k atm_h3_1y44olf atm_c8_2x1prs__oggzyc atm_g3_1jbyh58__oggzyc atm_fr_11a07z3__oggzyc dir dir-ltr"]')
        complete_data_list = [x.text for x in complete_data_list_primer]
        hosting_time = complete_data_list[-1]
    except:
        complete_data_list = np.nan
        hosting_time = "New host"
    return complete_data_list, hosting_time

def extract_reviews(driver, number_reviews):
    try:
        show_more_reviews(driver, number_reviews)
        all_reviews = scroll_reviews(driver)
    except:
        pass
    if len(all_reviews) < 1:
        try:
            all_reviews = driver.find_elements(By.XPATH, '//div[@class="r1bctolv atm_c8_1sjzizj atm_g3_1dgusqm atm_26_lfmit2_13uojos atm_5j_1y44olf_13uojos atm_l8_1s2714j_13uojos dir dir-ltr"]')
            all_reviews = [x.text for x in all_reviews]
        except:
            all_reviews = np.nan
    return all_reviews

def show_more_reviews(driver, number_reviews):
    try:
        more_reviews_button_xpath = f"//button[contains(text(), 'Show all {number_reviews} reviews')]"
        WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, more_reviews_button_xpath))
        ).click()
        sleep(2)
    except (TimeoutException, NoSuchElementException):
        pass

def scroll_reviews(driver):
    random_sleep = round(random.uniform(2, 3), 2)
    all_reviews = []

    try:
        review_popup = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "_17itzz4")))
        last_height = driver.execute_script("return arguments[0].scrollHeight", review_popup)
    except:
        return all_reviews

    while len(list(set(all_reviews))) < 100:
        driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", review_popup)
        sleep(random_sleep)
        reviews_list = driver.find_elements(By.XPATH, "//div[@class='r1bctolv atm_c8_1sjzizj atm_g3_1dgusqm atm_26_lfmit2_13uojos atm_5j_1y44olf_13uojos atm_l8_1s2714j_13uojos dir dir-ltr']")
        all_reviews.extend([review.text for review in reviews_list if len(review.text) > 3])
        sleep(random_sleep)
        new_height = driver.execute_script("return arguments[0].scrollHeight", review_popup)
        if new_height == last_height:
            break
        last_height = new_height
    return list(set(all_reviews))

In [48]:
def scraping_urls(city_url):
    # WebDriver
    options = webdriver.ChromeOptions()
    options.add_argument("--disable-search-engine-choice-screen")
    options.add_argument("--headless")
    options.add_argument('--lang=en')
    options.add_argument('--disable-geolocation')
    driver = webdriver.Chrome(options=options)

    # Airbnb Nottingham 
    # url = 'https://www.airbnb.com/s/Nottingham--England--United-Kingdom/homes'
    driver.get(city_url)

    all_urls: list = []
    all_prices: list = []
    # Scraping para la pagina en la que está
    def scrape_urls():
        sleep(8) # espera a que carge
        listings = driver.find_elements(By.XPATH, '//a[contains(@href, "/rooms/")]')
        urls = [listing.get_attribute('href') for listing in listings]
        sleep(3)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight * 0.7);")  # Baja al 90% de la página
        sleep(2)
        prices_primer = driver.find_elements(By.XPATH, '//span[@class="_11jcbg2"]')
        # class="_1jo4hgw" class="_1qgfaxb1"
        prices = [price.text for price in prices_primer]
        return urls, prices

    # Loop para todas las paginas
    while True:
        page_urls, price_list = scrape_urls() # Esto funciona porque cada vez que vamos a una pagina nueva la url principal no cambia
        all_urls.extend(page_urls)
        all_prices.extend(price_list)
        # hacer click en el boton "next"
        # break
        
        try:
            # next_button = driver.find_element(By.XPATH, '//a[@aria-label="Next"]')
            # driver.execute_script("arguments[0].scrollIntoView(true);", next_button)  # Desplaza hasta el botón "Next"
            next_button = driver.find_element(By.XPATH, '//a[@aria-label="Next"]')
            driver.execute_script("arguments[0].click();", next_button)  # Forzar clic en el botón "Next" usando JavaScript

            sleep(2)  # Espera a que el desplazamiento se complete
            # next_button.click()  # Haz clic en el botón "Next"
        except NoSuchElementException:
            # print(len(set(all_prices)), set(all_prices))
            break

    all_urls = list(set(all_urls))
    all_prices = list(set(all_prices))
    return all_urls, all_prices

In [49]:
url_nottingham = 'https://www.airbnb.com/s/Nottingham--England--United-Kingdom/homes'
url_san_francisco = "https://www.airbnb.com/s/San-Francisco--California--United-States/homes"
urls, prices_SF = scraping_urls(url_san_francisco)

168 {'€ 224 ', '€ 197 ', '€ 314 ', '€ 320 ', '€ 201 ', '€ 260 ', '€ 124 ', '€ 235 ', '€ 265 ', '€ 147 ', '€ 106 ', '€ 351 ', '€ 90 ', '€ 319 ', '€ 115 ', '€ 151 ', '€ 256 ', '€ 134 ', '€ 273 ', '€ 95 ', '€ 186 ', '€ 128 ', '€ 243 ', '€ 80 ', '€ 233 ', '€ 176 ', '€ 169 ', '€ 59 ', '€ 295 ', '€ 91 ', '€ 69 ', '€ 89 ', '€ 381 ', '€ 126 ', '€ 315 ', '€ 349 ', '€ 258 ', '€ 146 ', '€ 157 ', '€ 311 ', '€ 167 ', '€ 114 ', '€ 412 ', '€ 131 ', '€ 212 ', '€ 108 ', '€ 168 ', '€ 143 ', '€ 77 ', '€ 343 ', '€ 196 ', '€ 180 ', '€ 136 ', '€ 105 ', '€ 173 ', '€ 292 ', '€ 129 ', '€ 110 ', '€ 100 ', '€ 263 ', '€ 342 ', '€ 162 ', '€ 96 ', '€ 171 ', '€ 154 ', '€ 140 ', '€ 888 ', '€ 218 ', '€ 120 ', '€ 553 ', '€ 121 ', '€ 149 ', '€ 399 ', '€ 153 ', '€ 261 ', '€ 113 ', '€ 88 ', '€ 225 ', '€ 161 ', '€ 144 ', '€ 71 ', '€ 207 ', '€ 1,061 ', '€ 74 ', '€ 220 ', '€ 93 ', '€ 519 ', '€ 232 ', '€ 255 ', '€ 156 ', '€ 130 ', '€ 175 ', '€ 170 ', '€ 266 ', '€ 294 ', '€ 226 ', '€ 123 ', '€ 187 ', '€ 94 ', '€ 373 ', '€ 442 

In [ ]:
prices_SF

In [15]:
url_guest_favorite = "https://www.airbnb.com/rooms/17618950?adults=1&category_tag=Tag%3A8678&children=0&enable_m3_private_room=true&infants=0&pets=0&photo_id=1258742482&search_mode=regular_search&check_in=2024-10-30&check_out=2024-11-04&source_impression_id=p3_1727423252_P3uSk5kdl3FQVL7_&previous_page_section_name=1000&federated_search_id=29bac5dc-6e88-4a96-9f0f-fae6c05a3597"
url_no_guest_favorite = "https://www.airbnb.com/rooms/1226376809052320405?adults=1&children=0&enable_m3_private_room=true&infants=0&pets=0&search_mode=regular_search&check_in=2024-10-22&check_out=2024-10-25&source_impression_id=p3_1727424454_P3wkNr7x-Ud7bO04&previous_page_section_name=1000&federated_search_id=f573770f-c781-45af-a464-dd123435c95f"
url_new = "https://www.airbnb.com/rooms/27701970?adults=1&children=0&enable_m3_private_room=true&infants=0&pets=0&search_mode=regular_search&check_in=2024-11-14&check_out=2024-11-19&source_impression_id=p3_1727424233_P3x3d1pTlPM2C8Zb&previous_page_section_name=1000&federated_search_id=b1b3ca1e-4c9f-42f9-9a32-d8fbd3b10aa3"
url_2 = "https://www.airbnb.com/rooms/5413930?adults=1&children=0&enable_m3_private_room=true&infants=0&pets=0&search_mode=regular_search&check_in=2024-12-04&check_out=2024-12-09&source_impression_id=p3_1727425011_P3toaJxneSLX1aej&previous_page_section_name=1000&federated_search_id=f573770f-c781-45af-a464-dd123435c95f"
url_problema = "https://www.airbnb.com/rooms/41538482?adults=1&category_tag=Tag%3A8678&children=0&enable_m3_private_room=true&infants=0&pets=0&photo_id=1076432064&search_mode=regular_search&check_in=2024-10-30&check_out=2024-11-04&source_impression_id=p3_1727690290_P34aOaVgGAgAFFpB&previous_page_section_name=1000&federated_search_id=ad80992d-5bc2-4687-b24d-0d1f6c79cb68"
url_3 = "https://www.airbnb.com/rooms/1199915616176105267?adults=1&category_tag=Tag%3A8678&children=0&enable_m3_private_room=true&infants=0&pets=0&photo_id=1955356580&search_mode=regular_search&check_in=2024-11-04&check_out=2024-11-09&source_impression_id=p3_1727690322_P3f_su6KFa0YS6tG&previous_page_section_name=1000&federated_search_id=04476faa-a009-4449-8a1e-e7dfbf28f27a"
url_900_reviews = "https://www.airbnb.com/rooms/14996188?adults=1&children=0&enable_m3_private_room=true&infants=0&pets=0&search_mode=regular_search&check_in=2024-11-09&check_out=2024-11-14&source_impression_id=p3_1727696700_P31z7erJwV7ODI40&previous_page_section_name=1000&federated_search_id=64ab16be-1707-46e7-bce7-955ee4d6bd89"
url_4 = "https://www.airbnb.com/rooms/47476769?adults=1&children=0&enable_m3_private_room=true&infants=0&pets=0&search_mode=regular_search&check_in=2024-11-18&check_out=2024-11-23&source_impression_id=p3_1727697208_P3G-OrMWYXvW785E&previous_page_section_name=1000&federated_search_id=64ab16be-1707-46e7-bce7-955ee4d6bd89"
url_5 = "https://www.airbnb.com/rooms/3712191?adults=1&children=0&enable_m3_private_room=true&infants=0&pets=0&search_mode=regular_search&check_in=2024-10-28&check_out=2024-11-02&source_impression_id=p3_1727697705_P3VtOyl8YBojuvt-&previous_page_section_name=1000&federated_search_id=a3da65d1-f655-4932-b203-00d7ea5e1447"
url_no_reviews = "https://www.airbnb.com/rooms/1254652310065298533?adults=1&children=0&enable_m3_private_room=true&infants=0&pets=0&search_mode=regular_search&check_in=2024-10-01&check_out=2024-10-06&source_impression_id=p3_1727697624_P3xRvRTXu_LjJG4n&previous_page_section_name=1000&federated_search_id=33e9c87d-3ce4-41b0-9447-3f9b75712890"

otro = "https://www.airbnb.com/rooms/51581993?adults=1&children=0&enable_m3_private_room=true&infants=0&pets=0&search_mode=regular_search&check_in=2024-10-06&check_out=2024-10-11&source_impression_id=p3_1728231307_P3cf2qlMttmDdpx8&previous_page_section_name=1000&federated_search_id=903e0111-f821-4e6c-8bb2-9862637a26a2"
otro2 = "https://www.airbnb.com/rooms/1206601525586537688?adults=1&children=0&enable_m3_private_room=true&infants=0&pets=0&search_mode=regular_search&check_in=2024-10-09&check_out=2024-10-14&source_impression_id=p3_1728231176_P3p7_QPcWY5ktkLY&previous_page_section_name=1000&federated_search_id=86e37045-ce12-4340-8bc0-bfdbe2294f18"
guest_favorite, rating, number_reviews, type_host, hosting_time, all_reviews, title, complete_data_list = scraping_airbnb(url_guest_favorite)

print(f"guest_favorite: {guest_favorite}")
print(f"rating: {rating}")
print(f"number_of_reviews: {number_reviews}")
print(f"type_host: {type_host}")
print(f"hosting_time: {hosting_time}")
# print(f"Price: {price}")
print(all_reviews)
print(f"title: {title}")
print(len(all_reviews))
print(f"complete_list: {complete_data_list}")

print(f"guest_favorite: {type(guest_favorite)}")
print(f"rating: {type(rating)}")
print(f"number_of_reviews: {type(number_reviews)}")
print(f"type_host: {type(type_host)}")
print(f"hosting_time: {type(hosting_time)}")
# print(f"Price: {type(price)}")
print(type(all_reviews))
# print(len(all_reviews))

guest_favorite: True
rating: Rated 4.98 out of 5 stars.
4.98
number_of_reviews: 294
type_host: 1 bed
hosting_time: 1 bed · Shared bathroom
['The house of Kathryn is wounderful. I really loved every minute I spent there. It is very cosy and confortable. The room is lovely and there is also a garden around the house.\nKathryn was really an Excellent host, she made me feel home.\nThe place is located in a very nice aerea very close to the city centre and very well connected by bus. Close by there are nice restaurant and shops.', 'excellent stay from start to finish, would highly recommend.', 'I had a very good nights sleep which is the most important thing for me. The house is a short bus ride from central Nottingham.\nKat was very good and quick to reply to any questions.\nI would definitely stay again.', 'A kind and hospitable host, I felt at home in her lovely home, provides other help, great atmosphere, I will be happy to come there again.', "Kathryn was an awesome host! I've never us

In [52]:
df = pd.DataFrame(columns=['title', 'guest_favorite', 'rating', 'number_reviews', 'type_host', 'hosting_time', 'price', 'all_reviews', 'complete_data_list'])

In [ ]:
# df = pd.DataFrame()
n = 0
for url, price in zip(urls, prices_SF):
    n = n + 1
    print(f"{n} {price} {url}")
    guest_favorite, rating, number_reviews, type_host, hosting_time, all_reviews, title, complete_data_list = scraping_airbnb(url) # price,
    new_row = pd.DataFrame({
                'title' : [title],
                'guest_favorite': [guest_favorite],
                'rating': [rating],
                'number_reviews': [number_reviews],
                'type_host': [type_host],
                'hosting_time': [hosting_time],
                'price': [price],
                'all_reviews': [all_reviews],
                'complete_data_list' : [complete_data_list]
            })
    df = pd.concat([df, new_row], ignore_index=True)
    # if n > 5:
    #     break

# df.to_csv("SanFrancisco1.csv")
df